In [1]:
import pypelines.ts_forecast_pipeline as pipe
from pypelines import utils
import pandas as pd
airline = pd.read_csv("pypelines/datasets/timeseries_forecast/airline.csv")
longley = pd.read_csv("pypelines/datasets/timeseries_forecast/longley.csv")

In [2]:
del longley['Obs']

In [3]:
longley

,TOTEMP,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,60323,83.0,234289,2356,1590,107608,1947
1,61122,88.5,259426,2325,1456,108632,1948
2,60171,88.2,258054,3682,1616,109773,1949
3,61187,89.5,284599,3351,1650,110929,1950
4,63221,96.2,328975,2099,3099,112075,1951
5,63639,98.1,346999,1932,3594,113270,1952
6,64989,99.0,365385,1870,3547,115094,1953
7,63761,100.0,363112,3578,3350,116219,1954
8,66019,101.2,397469,2904,3048,117388,1955
9,67857,104.6,419180,2822,2857,118734,1956


In [4]:
tsf = pipe.TSForecastPipeline(data = "airline",models=['SARIMAX'],
                 date_column = 'Date',
                 date_format = "%Y-%m",
                 frequency = "M",
                 forecast_horizon = [1,2,3])

In [5]:
tsf = pipe.TSForecastPipeline(data = "longley",models=['Trend Forecast'],
                 date_column = 'YEAR',
                 date_format = "%Y",
                 target_column='TOTEMP',
                 frequency = "Y",
                 forecast_horizon = [1,2,3],
                 use_exogeneous_vars='True')

In [6]:
tsf.code_to_clipboard()

In [8]:

from sktime import *
from sklearn.metrics import mean_squared_error


import pandas as pd
import numpy as np

target_col='TOTEMP'
date_col = 'YEAR'


features = list(longley.columns.drop(target_col))
y = longley[[date_col,target_col]]
y_train_preprocessed = y
y_train_preprocessed[date_col] = pd.to_datetime(y_train_preprocessed[date_col],format = "%Y")
y_train_preprocessed = y_train_preprocessed.set_index(date_col)
y_train_preprocessed.index = y_train_preprocessed.index.to_period(freq = "Y")
X = longley[features]
x_train_preprocessed = X
x_train_preprocessed[date_col] = pd.to_datetime(x_train_preprocessed[date_col],format = "%Y")
x_train_preprocessed = x_train_preprocessed.set_index(date_col)
x_train_preprocessed.index = x_train_preprocessed.index.to_period(freq = "Y")

model_comparison_list = []

##### End of Data Processing Pipeline #####


##### Model Pipeline for Trend Forecast #####

from sktime.forecasting.trend import TrendForecaster
from sktime.forecasting.model_selection import ExpandingWindowSplitter,ForecastingGridSearchCV,ExpandingWindowSplitter
from sklearn.metrics import mean_squared_error,make_scorer,r2_score,explained_variance_score
import matplotlib.pyplot as plt
cv = ExpandingWindowSplitter(fh=[1, 2, 3])

TrendForecaster_param_grid = {
}


TrendForecaster_model = TrendForecaster()

TrendForecaster_gscv = ForecastingGridSearchCV(
    forecaster=TrendForecaster_model,
    param_grid=TrendForecaster_param_grid,
    cv=cv,
    verbose=1)


TrendForecaster_gscv.fit(y=y_train_preprocessed,X=x_train_preprocessed)


# get the prediction on the test data
TrendForecaster_y_pred = TrendForecaster_gscv.predict(fh=[1, 2, 3])
##### End of Model Pipeline for Trend Forecast #####
##### Model Comparison #####



/var/folders/0s/h092x9t92kl4ww1sxzs5wf4h0000gn/T/ipykernel_7636/1649165995.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_preprocessed[date_col] = pd.to_datetime(y_train_preprocessed[date_col],format = "%Y")
/var/folders/0s/h092x9t92kl4ww1sxzs5wf4h0000gn/T/ipykernel_7636/1649165995.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_preprocessed[date_col] = pd.to_datetime(x_train_preprocessed[date_col],format = "%Y")


Fitting 4 folds for each of 1 candidates, totalling 4 fits
